In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select   # seleccion de un dropdown


import time

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


import asyncio

from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from unidecode import unidecode 
import concurrent.futures

In [3]:
driver = webdriver.Chrome()

# Open the website
url = 'https://www.mundodeportivo.com/resultados/futbol/laliga'
driver.get(url)

In [3]:
aceptar = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')

aceptar.click()

In [4]:
aceptar_1 = driver.find_element(By.XPATH, '//*[@id="gg-alert-buttons"]/a[2]')

aceptar_1.click()

In [5]:
estadisticas = driver.find_element(By.XPATH, '/html/body/header/div[5]/div[1]/div/div/ul/li[6]/a')

estadisticas.click()

In [6]:
dropdown = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]')
dropdown.click()

In [7]:
fecha = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]/div/a[2]')
fecha.click()

In [8]:
goleadores = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[1]/div/a')
goleadores.click()

In [22]:
tabla_elemento = driver.find_element(By.XPATH, '//div[3]/div/div[1]/div[2]/div[1]/div/table/tbody')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [23]:
df = pd.DataFrame(data, columns=None)

In [24]:
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nRobert Lewandowski,23,Escudo Barcelona
2,2.\nKarim Benzema,19,Escudo Real Madrid
3,3.\nJoselu,16,Escudo Espanyol
4,4.\nAntoine Griezmann,15,Escudo Atlético de Madrid


In [46]:
nombres_columnas = df.iloc[0]

goleadores = df[1:].reset_index(drop=True)
goleadores.columns = nombres_columnas

In [47]:
goleadores.head()

,NOMBRE,TOTAL,None
0,1.\nRobert Lewandowski,23,Barcelona
1,2.\nKarim Benzema,19,Real Madrid
2,3.\nJoselu,16,Espanyol
3,4.\nAntoine Griezmann,15,Atlético de Madrid
4,5.\nVedat Muriqi,15,Mallorca


In [48]:
goleadores['NOMBRE'] = goleadores['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])

In [49]:
goleadores['NOMBRE'] = goleadores['NOMBRE'].apply(unidecode)

In [50]:
goleadores['NOMBRE'] = goleadores['NOMBRE'].apply(lambda x: x.title())

In [51]:
goleadores.columns

Index(['NOMBRE', 'TOTAL', None], dtype='object', name=0)

In [52]:
goleadores.columns = ['Jugador', 'Goles_totales', 'Equipo']

In [53]:
goleadores['Equipo'] = goleadores['Equipo'].str.replace('Escudo', '')


In [65]:
goleadores['Equipo'] = asistencias['Equipo'].apply(unidecode)

In [66]:
goleadores.head()

,Jugador,Goles_totales,Equipo
0,Robert Lewandowski,23,Atletico de Madrid
1,Karim Benzema,19,Real Madrid
2,Joselu,16,Real Sociedad
3,Antoine Griezmann,15,Real Madrid
4,Vedat Muriqi,15,Almeria


In [44]:
asistencias = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[2]/div/a')
asistencias.click()

In [45]:
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[2]/div/table')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [55]:
df = pd.DataFrame(data, columns=None)

In [56]:
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nAntoine Griezmann,16,Escudo Atlético de Madrid
2,2.\nVinícius Júnior,9,Escudo Real Madrid
3,3.\nMikel Merino,9,Escudo Real Sociedad
4,4.\nRodrygo,8,Escudo Real Madrid


In [57]:
nombres_columnas = df.iloc[0]

asistencias = df[1:].reset_index(drop=True)
asistencias.columns = nombres_columnas

In [59]:
asistencias.head()

,NOMBRE,TOTAL,None
0,1.\nAntoine Griezmann,16,Escudo Atlético de Madrid
1,2.\nVinícius Júnior,9,Escudo Real Madrid
2,3.\nMikel Merino,9,Escudo Real Sociedad
3,4.\nRodrygo,8,Escudo Real Madrid
4,5.\nLucas Robertone,7,Escudo Almería


In [61]:
asistencias['NOMBRE'] = asistencias['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])
asistencias['NOMBRE'] = asistencias['NOMBRE'].apply(unidecode)
asistencias['NOMBRE'] = asistencias['NOMBRE'].apply(lambda x: x.title())
asistencias.columns = ['Jugador', 'Asistencias_totales', 'Equipo']
asistencias['Equipo'] = asistencias['Equipo'].apply(unidecode)
asistencias['Equipo'] = asistencias['Equipo'].str.replace('Escudo', '')

In [64]:
asistencias.head()

,Jugador,Asistencias_totales,Equipo
0,Antoine Griezmann,16,Atletico de Madrid
1,Vinicius Junior,9,Real Madrid
2,Mikel Merino,9,Real Sociedad
3,Rodrygo,8,Real Madrid
4,Lucas Robertone,7,Almeria


In [67]:
porteros_goles = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[3]/div/a')
porteros_goles.click()

In [68]:
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[3]/div/table')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [69]:
df = pd.DataFrame(data, columns=None)

In [70]:
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nEdgar Badia,63,Escudo Elche
2,2.\nFernando Martínez,61,Escudo Almería
3,3.\nStole Dimitrievski,49,Escudo Rayo Vallecano
4,4.\nConan Ledesma,48,Escudo Cádiz


In [71]:
nombres_columnas = df.iloc[0]

goles_recibidos = df[1:].reset_index(drop=True)
goles_recibidos.columns = nombres_columnas

In [72]:
goles_recibidos['NOMBRE'] = goles_recibidos['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])
goles_recibidos['NOMBRE'] = goles_recibidos['NOMBRE'].apply(unidecode)
goles_recibidos['NOMBRE'] = goles_recibidos['NOMBRE'].apply(lambda x: x.title())
goles_recibidos.columns = ['Jugador', 'Goles_recibidos', 'Equipo']
goles_recibidos['Equipo'] = goles_recibidos['Equipo'].apply(unidecode)
goles_recibidos['Equipo'] = goles_recibidos['Equipo'].str.replace('Escudo', '')

In [73]:
goles_recibidos.head()

,Jugador,Goles_recibidos,Equipo
0,Edgar Badia,63,Elche
1,Fernando Martinez,61,Almeria
2,Stole Dimitrievski,49,Rayo Vallecano
3,Conan Ledesma,48,Cadiz
4,David Soria,45,Getafe


In [76]:
porteros_paradas=driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[4]/div/a')
porteros_paradas.click()

In [77]:
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[4]/div/table')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [78]:
df = pd.DataFrame(data, columns=None)
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nEdgar Badia,139,Escudo Elche
2,2.\nConan Ledesma,135,Escudo Cádiz
3,3.\nFernando Martínez,133,Escudo Almería
4,4.\nDavid Soria,124,Escudo Getafe


In [79]:
nombres_columnas = df.iloc[0]

paradas = df[1:].reset_index(drop=True)
paradas.columns = nombres_columnas

In [80]:
paradas['NOMBRE'] = paradas['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])
paradas['NOMBRE'] = paradas['NOMBRE'].apply(unidecode)
paradas['NOMBRE'] = paradas['NOMBRE'].apply(lambda x: x.title())
paradas.columns = ['Jugador', 'Paradas_totales', 'Equipo']
paradas['Equipo'] = paradas['Equipo'].apply(unidecode)
paradas['Equipo'] = paradas['Equipo'].str.replace('Escudo', '')


In [81]:
paradas.head()

,Jugador,Paradas_totales,Equipo
0,Edgar Badia,139,Elche
1,Conan Ledesma,135,Cadiz
2,Fernando Martinez,133,Almeria
3,David Soria,124,Getafe
4,Jordi Masip,105,Real Valladolid


In [82]:
tarjetas_amarillas=driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[5]/div/a')
tarjetas_amarillas.click()

In [83]:
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[5]/div/table')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [84]:
df = pd.DataFrame(data, columns=None)
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nFede San Emeterio,14,Escudo Cádiz
2,2.\nÓscar Gil,14,Escudo Espanyol
3,3.\nÍñigo Ruíz de Galarreta,12,Escudo Mallorca
4,4.\nAlejandro Catena,12,Escudo Rayo Vallecano


In [85]:
nombres_columnas = df.iloc[0]

tarjetas_amarillas = df[1:].reset_index(drop=True)
tarjetas_amarillas.columns = nombres_columnas

In [86]:
tarjetas_amarillas['NOMBRE'] = tarjetas_amarillas['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])
tarjetas_amarillas['NOMBRE'] = tarjetas_amarillas['NOMBRE'].apply(unidecode)
tarjetas_amarillas['NOMBRE'] = tarjetas_amarillas['NOMBRE'].apply(lambda x: x.title())
tarjetas_amarillas.columns = ['Jugador', 'Tarjetas_amarillas', 'Equipo']
tarjetas_amarillas['Equipo'] = tarjetas_amarillas['Equipo'].apply(unidecode)
tarjetas_amarillas['Equipo'] = tarjetas_amarillas['Equipo'].str.replace('Escudo', '')
tarjetas_amarillas.head()

,Jugador,Tarjetas_amarillas,Equipo
0,Fede San Emeterio,14,Cadiz
1,Oscar Gil,14,Espanyol
2,Inigo Ruiz De Galarreta,12,Mallorca
3,Alejandro Catena,12,Rayo Vallecano
4,Martin Zubimendi,12,Real Sociedad


In [87]:
tarjetas_rojas=driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[6]/div/a')
tarjetas_rojas.click()

In [88]:
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div[6]/div/table')

# Find all rows in the table
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")

# Initialize an empty list to store the data
data = []

# Loop through each row
for fila in filas:
    # Find all columns in the row
    columnas = fila.find_elements(By.TAG_NAME, "td")
    
    # Extract text data from each column
    fila_data = [columna.text for columna in columnas]
    
    # Find the image element in the first column
    image_element_xpath = './/td[1]/div/a/img'
    try:
        image_element = fila.find_element(By.XPATH, image_element_xpath)
        # Extract alt attribute from the image
        alt_text = image_element.get_attribute("alt")
        fila_data.append(alt_text)
    except:
        pass
    
    # Append the row data to the main data list
    data.append(fila_data)

In [89]:
df = pd.DataFrame(data, columns=None)
df.head()

,0,1,2
0,NOMBRE,TOTAL,None
1,1.\nIza Carcelén,3,Escudo Cádiz
2,2.\nFlorian Lejeune,3,Escudo Rayo Vallecano
3,3.\nGermán Pezzella,3,Escudo Real Betis
4,4.\nLuiz Felipe,3,Escudo Real Betis


In [90]:
nombres_columnas = df.iloc[0]

tarjetas_rojas = df[1:].reset_index(drop=True)
tarjetas_rojas.columns = nombres_columnas

In [92]:
tarjetas_rojas['NOMBRE'] = tarjetas_rojas['NOMBRE'].apply(lambda x: x.split('\n', 1)[-1])
tarjetas_rojas['NOMBRE'] = tarjetas_rojas['NOMBRE'].apply(unidecode)
tarjetas_rojas['NOMBRE'] = tarjetas_rojas['NOMBRE'].apply(lambda x: x.title())
tarjetas_rojas.columns = ['Jugador', 'Tarjetas_rojas', 'Equipo']
tarjetas_rojas['Equipo'] = tarjetas_rojas['Equipo'].apply(unidecode)
tarjetas_rojas['Equipo'] = tarjetas_rojas['Equipo'].str.replace('Escudo', '')
tarjetas_rojas.head()

,Jugador,Tarjetas_rojas,Equipo
0,Iza Carcelen,3,Cadiz
1,Florian Lejeune,3,Rayo Vallecano
2,German Pezzella,3,Real Betis
3,Luiz Felipe,3,Real Betis
4,Marcos Acuna,3,Sevilla
